In [1]:
import os
import glob
import psycopg2
#import pandas
import json
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import io
#from sql_queries import *

In [2]:
def create_StagingTables():
        
    StgSong_table_drop = ("""DROP TABLE if exists stg_songfiles;""")
    StgLog_table_drop = ("""DROP TABLE if exists stg_logfiles;""")
    StgSong_table_create = ("""CREATE TABLE IF NOT EXISTS stg_songfiles (num_songs int, artist_id varchar, artist_latitude real,  artist_longitude real, artist_location varchar, artist_name varchar,  song_id varchar, title varchar, duration real,  year int );""")
    StgLog_table_create = ("""CREATE TABLE IF NOT EXISTS stg_logfiles (artist varchar, auth varchar,firstName varchar, gender char(2),itemInSession int,lastName varchar,length real,level varchar,location varchar,    method varchar,    page varchar, registration real, sessionId int,  song varchar, status int, ts bigint,userAgent varchar, userId varchar);""")
    
    drop_Stage_tables = [StgSong_table_drop, StgLog_table_drop]
    create_Stage_tables = [StgSong_table_drop, StgLog_table_drop,StgSong_table_create, StgLog_table_create]
    
    # connect to sparkify database
    conn = psycopg2.connect("dbname=sparkifydb user=postgres password=Hinati2015 host=localhost port=5433")    
    cur = conn.cursor()
    
    #Activate connection server
    cur = conn.cursor() 
    
    for query in drop_Stage_tables:
        cur.execute(query)
        conn.commit()  
    
    for query in create_Stage_tables:
        cur.execute(query)
        conn.commit()    
    
    conn.close()

In [3]:
def process_song_file(cur, filepath):
    
    # open/read song file and write to dataframe
    df=pd.read_json(filepath, lines='True')   
        
    # capture values from json and create list
    song_dataList = df[['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude', 'artist_location', 'artist_name', 'song_id', 'title', 'duration', 'year']].values.tolist()[0]           
    
    # insert into staging table for analysis
    cur.execute("INSERT INTO stg_songfiles (num_songs, artist_id, artist_latitude,  artist_longitude, artist_location, artist_name, song_id, title, duration,  year) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", song_dataList)
    
    # insert song record
    cur.execute("INSERT INTO dimsong (song_id, title, artist_id, yearz, duration) Select distinct song_id, title, artist_id, year, duration FROM stg_songfiles ");
    
    #cur.execute(song_table_insert, song_data)
    
    # insert artist record    
    cur.execute("INSERT INTO dimartist (artist_id,NAME,location,latitude,longitude) \
                SELECT  distinct artist_id, artist_name, artist_location, artist_latitude, artist_longitude  FROM stg_songfiles order by 1");
                
    ##cur.execute(artist_table_insert, artist_data)
    

In [4]:
def process_log_file(cur, filepath):
    # open log file and write to dataframe
    df=pd.read_json(filepath, lines='True')

    # capture values from json and create list
    log_dataList = df[['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId' ,  'song' , 'status', 'ts', 'userAgent', 'userId']].values.tolist()[0]    
        
    # insert into staging table for analysis
    cur.execute("INSERT into stg_logfiles (artist, auth, firstName, gender ,itemInSession ,lastName ,length ,level ,location ,method ,page, registration, sessionId ,  song , status, ts ,userAgent, userId) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", log_dataList)
    
    # load Dimtime
    cur.execute("INSERT into dimtime (start_time, hour, day, week, month, year, weekday) \
    select TO_TIMESTAMP(ts), date_part('hour', CAST(TO_TIMESTAMP(ts) as timestamp)) as hour, date_part('day', CAST(TO_TIMESTAMP(ts) as timestamp)) as day, date_part('week', CAST(TO_TIMESTAMP(ts) as timestamp)) as week, date_part('month', CAST(TO_TIMESTAMP(ts) as timestamp)) as month, date_part('year', CAST(TO_TIMESTAMP(ts) as timestamp)) as year, date_part('week', CAST(TO_TIMESTAMP(ts) as timestamp)) as weekday from stg_logfiles");               
    
    # load Dimuser table    
    cur.execute("INSERT INTO dimUser (user_id, first_name, last_name, gender, level) SELECT distinct userid, firstName, lastName,gender, level  FROM stg_logfiles");   
    
    # load Dimuser table   
    cur.execute("INSERT into  factSongplays (start_time, sgkeyuser_id, level, sgkeysong_id, sgkeyartist_id, session_id, location, user_agent)                       select to_timestamp(lf.ts) as start_time,  du.sgkeyuserid as user_id,  lf.level as level,  ds.sgkeysong_id as song_id, da.sgkeyArtist_id as artist_id,lf.sessionID as sessionID,  lf.location as location, lf.userAgent as userAgent from stg_logfiles lf  left join dimUser du on du.user_id = lf.userid  left join dimSong ds on ds.title = lf.song  left join dimArtist da on da.name = lf.artist left join dimtime dt on dt.start_time = TO_TIMESTAMP(lf.ts)")

In [5]:
def process_data(cur, conn, filepath, func):
    
    #write all filenames to list; filter empty files (<0Bytes)
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        size = os.stat(filepath).st_size
        for f in files:
            if os.path.getsize(f) > 0:
                all_files.append(os.path.abspath(f))
                
    

    #print list
    print('')
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))
    print('')

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed {}'.format(i, num_files, datafile))

In [6]:
def main():
    
    # Drop/Create Stage Tables
    create_StagingTables()    
    
    conn = psycopg2.connect("dbname=sparkifydb user=postgres password=Hinati2015 host=localhost port=5433")
    conn.set_session(autocommit=True)
    
    # Activate Connection server
    cur = conn.cursor()
    
    process_data(cur, conn, filepath='C:\\Users\\Dyim\\Desktop\\Dayim\\01UdacityProject\\ProjectData\\SongData\\', func=process_song_file)
    process_data(cur, conn, filepath='C:\\Users\\Dyim\\Desktop\\Dayim\\01UdacityProject\\ProjectData\\LogData\\', func=process_log_file)

    conn.close()

In [7]:
main()


75 files found in C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\

1/75 files processed C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\A\A\A\TRAAAAW128F429D538.json
2/75 files processed C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\A\A\A\TRAAABD128F429CF47.json
3/75 files processed C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\A\A\A\TRAAADZ128F9348C2E.json
4/75 files processed C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\A\A\A\TRAAAEF128F4273421.json
5/75 files processed C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\A\A\A\TRAAAFD128F92F423A.json
6/75 files processed C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\A\A\A\TRAAAMO128F1481E7F.json
7/75 files processed C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\A\A\A\TRAAAMQ128F1460CD3.json
8/75 files processed C:\Users\Dyim\Desktop\Dayim\01UdacityProject\ProjectData\SongData\A\A\A\T